# $\mathbb{Z}_q\times\mathbb{Z}_{qw}$

In [1]:
from localFuncs import *

In [2]:
%load_ext Cython

In [3]:
%%cython

import itertools as it
import copy
import numpy as np
cimport numpy as np
cimport cython



@cython.cdivision(True)
cdef list get_orderToAdd(np.int q):
    """
    return a list of integers i in the order to add sets A_i to partiallyA
    """
    cdef int a
    cdef int i
    cdef list out
    
    out = [0] * q
    a = 1
    i = 1
    if q % 2 == 0:
        out[1] = q // 2
        i = 2
    while i < q:
        out[i] = a
        out[i + 1] = q - a
        a += 1
        i += 2
        
    return out



@cython.cdivision(True)
cdef list all_checks(int q):
    """
    return a list of tuples that define all of the weak (2,1)-sum-free set checks on the A_i
    """
    return list(it.combinations_with_replacement(range(q), r = 2))





@cython.cdivision(True)
cdef list mod_sort(set nums, int mod):
    """
    return a list of the elements specified in the set `nums` best sorted into an Arithmetic sequence in Z_`mod`
    """
    cdef list toSort
    cdef list sort
    cdef int breakPoint
    cdef int i
    cdef list out
    
    toSort = []
    for num in nums:
        if num < 0:
            toSort += [num + mod]
        else:
            toSort += [num]
    sort = sorted(toSort)
    breakPoint = len(sort)
    for i in range(len(sort) - 1):
        if sort[i+1] > sort[i] + 2:
            breakPoint = i + 1
    out = []
    for num in sort[breakPoint:len(sort)]:
        out += [num - mod]
    out += sort[:breakPoint]
    
    return sorted(out)











cdef class Node(object):
    cdef list partiallyA
    cdef list yetToBeChecked
    cdef int depth
    cdef int q
    cdef int w
    cdef list orderToAdd
    cdef tuple length

    def __init__(self, list partiallyA, list yetToBeChecked, 
                 int depth, int q, int w, tuple length):
        """
        construct a search node object
        """
        self.partiallyA = partiallyA
        self.yetToBeChecked = yetToBeChecked
        self.depth = depth
        self.orderToAdd = get_orderToAdd(q)
        self.q = q
        self.w = w
        self.length = length

        
        
    @cython.nonecheck(False)
    cpdef bint is_goal(self):
        """
        return boolean, has the Node's partially A set passed all 
        of the checks, is it weak (2,1)-sum-free
        """
        return len(self.yetToBeChecked) == 0
    
    
    
    cpdef void incrementDepth(self):
        """
        increment node depth by one
        """
        self.depth += 1

        
        
    cpdef set get_partiallyA(self, int i):
        """
        return the set in row `i` of partiallyA, A_i
        """
        return self.partiallyA[i]

    
    
    cpdef void set_partiallyA(self, int i, set arithSet):
        """
        set the `i`th row of partiallyA, A_i
        """
        self.partiallyA[i] = arithSet

        
        
    cpdef void remove_from_yetToBeChecked(self, tuple checkTup):
        """
        remove the check-determining tuple `checkTup` from the list
        of check tuples that have yet to be applied to partiallyA
        """
        self.yetToBeChecked.remove(checkTup)

        
        
    cpdef Node clone(self):        
        """
        return a deep clone of the Node
        """
        return Node(copy.copy(self.partiallyA), 
                    copy.copy(self.yetToBeChecked), 
                    self.depth, self.q, self.w, self.length)

    
    
    @cython.cdivision(True)
    cpdef bint check(self, tuple rowTup):
        """
        check PartiallyA using `rowTup`; use the row indices in `rowTup` 
        to ensure that these rows are weak (2,1)-sum-free
        """
        cdef int row1
        cdef int row2
        cdef int a
        cdef int b
        cdef set sumset
        
        row1 = rowTup[0]
        row2 = rowTup[1]
        
        sumset = set()
        for a in self.partiallyA[row1]:
            for b in self.partiallyA[row2]:
                if a != b or row1 != row2:
                    sumset.add((a + b) % (self.q * self.w))

        return len(sumset.intersection(self.partiallyA[(row1 + row2) % self.q])) == 0

    
    
    
    cpdef bint equals(self, Node node):
        """
        check content of this node and `node` to return boolean, is equal
        """
        cdef int i
        
        if self.depth != node.depth:
            return False
        if self.q != node.q:
            return False
        if self.w != node.w:
            return False
        if self.length != node.length:
            return False
        for i in range(self.q):
            if self.partiallyA[i] != node.get_partiallyA(i):
                return False
        return True
        
        
        
    @cython.cdivision(True)
    cpdef list expand(self):
        """
        return a list of this Node's children.
        Iterate through all of the possible starting values for the 
        starting values of the arithmetic set A_i 
        """
        cdef tuple checkTup
        cdef list children
        cdef list sortedArithSet
        cdef list children_rm
        cdef Node child1
        cdef Node child2
        cdef Node potentialChild
        cdef set arithSet

    
        if self.depth >= self.q:
            return []
            
        # iterate through every possible arithmetic sequence f
        # or the next A_i set to be added
        children = []
        for shift in range(self.q * self.w):            
            
            # creating the new child
            potentialChild = self.clone()
            potentialChild.incrementDepth()
            
            # add the arithmetic sequence to the child
            arithSet = set()
            for i in range(self.length[self.orderToAdd[self.depth]]):
                arithSet.add((shift + 2 * i) % (self.q * self.w))
                
            potentialChild.set_partiallyA(self.orderToAdd[self.depth], arithSet)

            # test to see if the child should be kept
            for checkTup in self.yetToBeChecked:
                
                # a check can be done if it hasn't been done already
                # and if it pertains to the sets that the child has)                                 
                if (len(potentialChild.get_partiallyA(checkTup[0])) != 0) and (len(potentialChild.get_partiallyA(checkTup[1])) != 0):
                    
                    # if check passes, remove the check tuple from yetToBeChecked
                    if potentialChild.check(checkTup):
                        potentialChild.remove_from_yetToBeChecked(checkTup)
                    
                    # otherwise, this potentialChild is no good, so go to next `shift`
                    else:
                        break

            # all the possible checks are good, so potentialChild is OK for now
            children.append(potentialChild)
            
         # remove duplicate children - I'm not sure if there are duplicate children
        children_rm = []
        for child1 in children:
            if any([child1.equals(child2) for child2 in children_rm]):
                next
            children_rm += [child1]
        return children
    
    
    
    
    def __str__(self):
        """
        return pretty format for set partiallyA
        """
        cdef str string
        cdef int i
        
        string = ""
        i = 0
        for arithSet in self.partiallyA:
            string += "{" + f"{i}" + "}" + " x {" 
            string += str(mod_sort(arithSet, self.q * self.w))[1:-1] + "}\n"
            i += 1
        return string
            

        
        
        
        
        
        
        
        

cdef class SetFinder(object):
    cdef int q
    cdef int w
    cdef int number
    cdef int i
    cdef bint found
    cdef list out
    cdef set arithSet

    
    def __init__(self, int q, int w, int number):
        """
        construct a finder object
        """
        self.q = q
        self.w = w
        self.found = 0;
        self.number = number

        
    cpdef int find(self, tuple length, bint verbose):
        """
        start a search by creating a root Node and calling 
        help_find() to do the depth-first recursive search
        """
        cdef Node root
        
        root = Node([set()] * self.q, all_checks(self.q), 0, self.q, self.w, length)
        return self.help_find(length, root, verbose)
    
    
    
    cpdef int help_find(self, tuple length, Node node, bint verbose):
        """
        do a depth-first recursive search 
        """
        cdef list children 
        cdef list out
        cdef int i
        cdef Node childNode
        
        children = node.expand()
        if node.is_goal():
            if verbose:
                print(str(node))
            else:
                out = []
                for i in range(self.q):
                    out += [mod_sort(node.get_partiallyA(i), self.q * self.w)[0]]
                print(out)
            self.found += 1
        elif children == []:
            return self.found
        else:
            for childNode in children:
                self.help_find(length, childNode, verbose)
                if self.found >= self.number:
                    return self.found
            return self.found

In [4]:
q = 6
w = 1

v = q * w // 3
t = (v + 1,) + (v,) * (q - 1)


display(Markdown(f"$t = {t}$ so going to check " + '{:,.0f}'.format((q*w)**(len(t))) + " / " + '{:,.0f}'.format(2**(q*q*w)) + " of the subsets "
                 + f"$A\\subseteq \\mathbb{{Z}}_{{{q}}}\\times \\mathbb{{Z}}_{{{q* w}}}$," 
                 + f" each of size $1 + {q}\\frac{{{q}\cdot{w}}}{{{3}}} = {sum(t)}$<br>"))



finder = SetFinder(q, w, 1)
finder.find(t, verbose=True)

$t = (3, 2, 2, 2, 2, 2)$ so going to check 46,656 / 68,719,476,736 of the subsets $A\subseteq \mathbb{Z}_{6}\times \mathbb{Z}_{6}$, each of size $1 + 6\frac{6\cdot1}{3} = 13$<br>

{0} x {1, 3, 5}
{1} x {0, 2}
{2} x {1, 3}
{3} x {0, 2}
{4} x {1, 3}
{5} x {0, 2}



1

---
#### $Z_6 \times Z_6$

```
{0} x {1, 3, 5}
{1} x {0, 2}
{2} x {1, 3}
{3} x {0, 2}
{4} x {1, 3}
{5} x {0, 2}
```

#### $Z_6 \times Z_{12}$

```
{0} x {1, 3, 5, 7, 9}
{1} x {0, 2, 4, 6}
{2} x {1, 3, 5, 7}
{3} x {0, 2, 4, 6}
{4} x {1, 3, 5, 7}
{5} x {0, 2, 4, 6}
```

#### $Z_6 \times Z_{18}$

```
{0} x {1, 3, 5, 7, 9, 11, 13}
{1} x {0, 2, 4, 6, 8, 10}
{2} x {1, 3, 5, 7, 9, 11}
{3} x {0, 2, 4, 6, 8, 10}
{4} x {1, 3, 5, 7, 9, 11}
{5} x {0, 2, 4, 6, 8, 10}
```

#### $Z_6 \times Z_{24}$
```
{0} x {1, 3, 5, 7, 9, 11, 13, 15, 17}
{1} x {0, 2, 4, 6, 8, 10, 12, 14}
{2} x {1, 3, 5, 7, 9, 11, 13, 15}
{3} x {0, 2, 4, 6, 8, 10, 12, 14}
{4} x {1, 3, 5, 7, 9, 11, 13, 15}
{5} x {0, 2, 4, 6, 8, 10, 12, 14}
```